# 多头注意力

In [1]:
import torch
from d2l import torch as d2l
from torch import nn
import math

In [2]:
def transpose_qkv(X, num_heads):
    """为了多注意力头的并行计算而变换形状"""
    # 输入X的形状:(batch_size，查询或者“键－值”对的个数，num_hiddens)
    # 输出X的形状:(batch_size，查询或者“键－值”对的个数，num_heads， num_hiddens/num_heads)
    X= X.reshape(X.shape[0], X.shape[1], num_heads, -1)
    # 输出X的形状:(batch_size，num_heads，查询或者“键－值”对的个数, num_hiddens/num_heads)
    X = X.permute(0, 2, 1, 3)
    # 最终输出的形状:(batch_size*num_heads,查询或者“键－值”对的个数, num_hiddens/num_heads)
    return X.reshape(-1, X.shape[2], X.shape[3])

In [3]:
#@save
def transpose_output(X, num_heads):
    """逆转transpose_qkv函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute(0, 2, 1, 3)
    return X.reshape(X.shape[0], X.shape[1], -1)

In [4]:
class MultiHeaderAttention(nn.Module):
    def __init__(self, key_size, query_size, value_size, num_hiddens, num_heads, dropout, bias=False) -> None:
        super().__init__()
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)
    
    def forward(self, queries, keys, values, valid_lens):
        # queries，keys，values的形状:
        # (batch_size，查询或者“键－值”对的个数，num_hiddens)
        # valid_lens　的形状:
        # (batch_size，)或(batch_size，查询的个数)
        # 经过变换后，输出的queries，keys，values　的形状:
        # (batch_size*num_heads，查询或者“键－值”对的个数，num_hiddens/num_heads)
        queries = transpose_qkv(self.W_q(queries), self.num_heads)
        keys = transpose_qkv(self.W_k(keys), self.num_heads)
        values = transpose_qkv(self.W_v(values), self.num_heads)
        
        if valid_lens is not None:
            # 将 valid_lens 重复 num_heads 次，因为每个注意力头都需要独立的 valid_lens
            # 注意：参数名是 repeats 而不是 repeat
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0
            )
        
        output = self.attention(queries, keys, values, valid_lens)
        
        output_cat = transpose_output(output, num_heads=self.num_heads)
        return self.W_o(output_cat)

In [5]:
num_hiddens, num_heads= 100, 5
attention = MultiHeaderAttention(num_hiddens, num_hiddens, num_hiddens, num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeaderAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [6]:
# 这里的 num_kvpairs 表示“键-值对”的数量
# 在 self-attention 里，query、key、value 都是向量序列，每个 kv 对有各自的 key 和 value 向量
# 为啥 value 中也有 num_kvpairs？因为键和值是成对存在的：每个 key 对应一个 value
# 在一般的序列建模中，key 和 value 是完全一样的 shape，不同角色而已
# 在 encoder-decoder attention 里 key/value 是encoder的输出，query是decoder的隐藏状态
batch_size, num_queries = 2, 4
num_kvpairs = 6  # “键-值对”的数量
valid_lens = torch.tensor([3, 2])

# X：query，形状 [batch_size, num_queries, num_hiddens]
# Y：key/value，形状 [batch_size, num_kvpairs, num_hiddens]
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))

# Y 既作为 key 也作为 value 传入，代表有 num_kvpairs 个“键-值对”
# 返回结果的 shape 是 [batch_size, num_queries, num_hiddens]
print(attention(X, Y, Y, valid_lens).shape)

torch.Size([2, 4, 100])